In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from test_memory import get_llm
from AzureConnection import embeddings


In [7]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


### vectorstore

In [8]:
loader = CSVLoader(file_path='OutdoorClothingCatalog_1000.csv')
docs = loader.load()

In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])


In [28]:
query = "list all shirts with sun protection"
result = index.query(query, llm=get_llm())
print(result)

Here are the shirts with sun protection:

1. Sun Shield Shirt: Lightweight shirt with UPF 50+ sun protection. Price: $45.00

2. Beach Shirt: Quick-dry with UPF 50+ sun protection. Price: $40.00

3. Mountain Shirt: Long sleeve with built-in UPF 30+ sun protection. Price: $55.00


### embedding II

In [41]:
embed = embeddings.embed_query("cat")
print(len(embed))

1536


In [42]:
print(embed[:5])

[-0.0070539116859436035, -0.01734057068824768, -0.009698242880403996, -0.03073945827782154, -0.012484360486268997]


### retriever

In [10]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)

query = "give me sun protection clothes"
result = db.similarity_search(query)
print(result)

[Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content='product_name: Sun Shield Shirt\ndescription: Lightweight shirt with UPF 50+ sun protection\nfeatures: UPF 50+\nprice: 45.00\ncategory: Shirts'), Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 5}, page_content='product_name: Mountain Shirt\ndescription: Long sleeve with built-in sun protection\nfeatures: UPF 30+\nprice: 55.00\ncategory: Shirts'), Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 2}, page_content='product_name: UV Blocker Hoodie\ndescription: Full coverage hoodie for extreme sun\nfeatures: UPF 50+\nprice: 60.00\ncategory: Hoodies'), Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 8}, page_content='product_name: Beach Shirt\ndescription: Quick-dry with sun protection\nfeatures: UPF 50+\nprice: 40.00\ncategory: Shirts')]


In [11]:
retriever = db.as_retriever()

In [12]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=get_llm(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

response = qa_stuff.invoke("give me sun protection clothes and give me output as a table in markdown format")
print(response)

{'query': 'give me sun protection clothes and give me output as a table in markdown format', 'result': "Sure! Here's a table of sun protection clothes in markdown format:\n\n```markdown\n| Product Name       | Description                          | Features  | Price | Category |\n|--------------------|--------------------------------------|-----------|-------|----------|\n| Sun Shield Shirt   | Lightweight shirt with UPF 50+ sun protection | UPF 50+  | $45.00| Shirts   |\n| Mountain Shirt     | Long sleeve with built-in sun protection | UPF 30+  | $55.00| Shirts   |\n| Safari Pants       | Convertible pants with sun protection | UPF 30+  | $75.00| Pants    |\n| Beach Shirt        | Quick-dry with sun protection        | UPF 50+  | $40.00| Shirts   |\n```", 'source_documents': [Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content='product_name: Sun Shield Shirt\ndescription: Lightweight shirt with UPF 50+ sun protection\nfeatures: UPF 50+\nprice: 45.00

![image](/image/embedding_stuff.png)